# SQL

## 1. Introducción
---

Durante esta actividad vamos a aprender los conceptos básicos de SQL. Vamos a aprender a insertar, eliminar y actualizar datos de una base de datos además de hacer consultas sencillas sobre ellos.

### 1.1 Requisitos

Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar

- Python 3
- Jupyter
- La librería `ipython-sql`. Para instalarla pueden hacerlo con `pip3`:
```
pip3 install ipython-sql
```
- SQLite3, que viene junto con la instalación de Python

**Importante**: si vas a correr este notebook de forma local, te recomendamos usar una versión de `sqlalchemy`inferior a la 1.4.0, por lo que tendrás que ejecutar la siguiente celda (descomenta las líneas antes de ejecutar):


In [ ]:
#pip3 uninstall sqlalchemy
#pip3 install --upgrade "sqlalchemy<1.4.0"

**Importante**: Si vas a correr este notebook en **google colab**, ejecuta la siguiente celda (descomenta las líneas antes de ejecutar y escribe `y` cuando la consola te lo pida):

In [ ]:
#!pip3 uninstall sqlalchemy
#!pip3 install --upgrade "sqlalchemy<1.4.0"

### 1.2 Outline

En esta actividad aprenderemos a:

- Crear y modificar tablas.
- Insertar, eliminar y actualizar datos.
- Crear llaves en las tablas.
- Hacer consultas que involucren selección, proyección, producto cruz y operaciones de conjuntos.

### 1.3 Esquema

Para esta actividad vamos a trabajar con el siguiente esquema:

- `Capitanes(cid INT PRIMARY KEY, cnombre VARCHAR(100), crating FLOAT, cedad INT)`
- `Botes(bid INT PRIMARY KEY, bnombre VARCHAR(100), bcolor VARCHAR(100))`
- `Reservas(cid INT, bid INT, fecha DATE, PRIMARY KEY(cid, bid))`

Que corresponde a capitanes que reservan ciertos botes. El `cid` y `bid` en la tabla `Reservas` proviene de las tablas `Capitanes` y `Botes` respectivamente.

## 2. Actividad
---

### 2.1 Crear tablas, eliminar tablas y llaves primarias

En esta actividad vamos a utilizar `ipython-sql` para tener acceso a SQL desde este _notebook_. Lo vamos a importar a continuación.

In [ ]:
%load_ext sql

Ahora vamos a crear una nueva base de datos para esta actividad. Esta base de datos se llamará `capitanes.db`.

In [ ]:
%sql sqlite:///capitanes.db

**Ojo**: cada vez que anteponemos `%sql` es porque esa línea corresponde a un comando a SQL que va a la base de datos en la que estamos trabajando (en este caso `capitanes.db`). Si queremos tener todo un bloque con instrucciones SQL tenemos que usar `%%sql` (habrán ejemplos de esto a lo largo del *notebook*).

Tu primera prueba será crear una base de datos de acuerdo el esquema de arriba. La sintaxis para crear tablas en SQL es la siguiente (si están trabajando desde el cliente, después de cada sentencia SQL debes poner un punto y coma `;`):

```SQL
CREATE TABLE <Nombre Tabla> (<atributo_1> tipo, ... , <attributo_N> tipo)
```

Por ejemplo, para crear la tabla de capitanes:

In [ ]:
%%sql 
DROP TABLE IF EXISTS Capitanes;
CREATE TABLE Capitanes(cid INT, cnombre VARCHAR(100), crating FLOAT, cedad INT);

En la celda anterior estamos eliminando la tabla si es que existe para luego crear una tabla según lo requerido por el esquema. Pero cuidado! Olvidamos agregar la llave primaria, por lo que vamos a **eliminar** la tabla y crearla de nuevo. Además, vamos a agregar un valor por defecto al atributo `crating`, que en este caso será 0.

In [ ]:
%%sql 
DROP TABLE Capitanes;
CREATE TABLE Capitanes(cid INT PRIMARY KEY, cnombre VARCHAR(100), crating FLOAT DEFAULT 0, cedad INT);

Otra opción es agregar la llave primaria al final de la instrucción `CREATE TABLE`. Para ver un ejemplo, vamos a crear la tabla de reservas.

In [ ]:
%%sql 
DROP TABLE IF EXISTS Reservas;

In [ ]:
%sql CREATE TABLE Reservas(cid INT, bid INT, fecha DATE, PRIMARY KEY(cid, bid));

In [ ]:
%%sql 
DROP TABLE IF EXISTS Reservas;
CREATE TABLE Reservas(cid INT, bid INT, fecha DATE, PRIMARY KEY(cid, bid));

En el caso anterior, tenemos una llave primaria compuesta. En general, para crear una tabla con llave primaria, lo señalamos al final, de la forma:

```SQL
CREATE TABLE <Nombre Tabla> (
    <atributo_1> tipo, ..., 
    <attributo_N> tipo, 
    PRIMARY KEY(<atributos separados por coma (,)>))
```

**1.** Ahora es tu turno. En la celda a continuación debes crear la tabla de botes. No olvides agregar su llave primaria.

In [ ]:
%sql

### 2.2 Tipos de datos

Al crear las tablas disponemos varios tipos de datos. Por ahora destacamos los siguientes:

- Caracteres (_Strings_):
  - `CHAR(20)`: _Strings_ de largo fijo.
  - `VARCHAR(20)`: _Strings_ de largo variable.
  
- Números:
  - `INT`
  - `FLOAT`
  - `SMALLINT`
  
- Tiempos y fechas:
  - `DATE`: fecha.
  - `TIME`: hora.
  - `TIMESTAMP`: fecha y hora.

### 2.3 Insertar, eliminar y modificar elementos en tablas

Si ejecutamos la consulta `SELECT * FROM Capitanes` notaremos que el resultado es vacío. Esto es evidente, ya que esta consulta me retorna todo lo que tengo en la tabla `Capitanes` y actualmente no hemos insertado nada.

In [ ]:
%sql SELECT * FROM Capitanes;

Para insertar valores, la forma básica es la siguiente:

```SQL
INSERT INTO <Nombre Tabla> 
VALUES (<valor atributo 1> , ..., <valor atributo N>)
```

Por ejemplo si ejecutamos la consulta:

```SQL
INSERT INTO Capitanes 
VALUES(1, 'Claudio', 0, 35)
```

estamos insertando un Capitán con `cid` 1, `cnombre` Claudio, `crating` 0 y `cedad` 35.

In [ ]:
%sql INSERT INTO Capitanes VALUES(1, 'Claudio', 0, 35)

Recordemos que habíamos señalado que la llave primaria de la tabla `Capitanes` era el `cid`. Veamos que pasa si intentamos insertar un capitán con el mismo `cid`.

In [ ]:
%sql INSERT INTO Capitanes VALUES(1, 'Claudio Bravo', 0, 35)

La consulta anterior debería haber arrojado un error. Así que vamos a cambiar el `cid`. Sin embargo, vamos a omitir algunos atributos.

In [ ]:
%sql INSERT INTO Capitanes(cid, cnombre) VALUES(13, 'Claudio Bravo')

In [ ]:
%sql SELECT * FROM Capitanes

Si te das cuenta, ahora tenemos dos tuplas en la tabla de capitanes. Como podrías haber esperado, dado que omitimos el valor para ciertos atributos en la segunda inserción, el `crating` tomó el valor _default_. Pero ojo! el atributo `cedad` tomó un valor **nulo**. Más adelante en el curso vamos a ahondar en esta temática y ver por qué los valores nulos pueden ser un problema. Ahora vamos a agregar una nueva tupla para hacer algunas consultas sencillas.

In [ ]:
%sql INSERT INTO Capitanes VALUES(23, 'Arturo Vidal', 8, 31)

In [ ]:
%sql SELECT * FROM Capitanes;

In [ ]:
%sql SELECT * FROM Capitanes WHERE cnombre='Claudio'

En la consulta anterior estamos filtrando por Capitanes donde su nombre sea `'Claudio'`. Si queremos que el nombre **contenga** `'Claudio'` usamos la instrucción `LIKE`:

In [ ]:
%sql SELECT * FROM Capitanes WHERE cnombre LIKE '%Claudio%'

Pueden notar que hemos añadido un `%` al _string_. Esto es porque antes y después de `Claudio` queremos permitir cualquier secuencia de caracteres. Para el `LIKE` tenemos dos instrucciones posibles:

- `%` que significa cualquier secuencia de caracteres.
- `_` que significa un caracter, pero **solamente** uno.

Como no estamos seguros de mantener a `Claudio` en la tabla `Capitanes` lo vamos a eliminar. Para eliminar tuplas lo hacemos con la instrucción `DELETE`. Veamos que pasa al ejecutar la siguiente consulta:

In [ ]:
%%sql
DELETE FROM Capitanes
WHERE cnombre LIKE '%Claudio%'

In [ ]:
%sql SELECT * FROM Capitanes

Lo que hicimos fue borrar de la tabla `Capitanes` todas las tuplas que satisfacen la condición en el `WHERE` de la consulta. Para actualizar una tupla usamos la instrucción `UPDATE`, como veremos a continuación:

In [ ]:
%sql UPDATE Capitanes SET cnombre='King Arturo' WHERE cid=23

In [ ]:
%sql SELECT * FROM Capitanes

### 2.4 Modificando tablas

En SQL es posible modificar tablas. Por ejemplo si quisieramos eliminar el atributo `crating` podríamos ejecutar la consulta:

```SQL
ALTER TABLE Capitanes DROP COLUMN crating
```

O agregar una columna, como por ejemplo:

```SQL
ALTER TABLE Capitanes ADD COLUMN rut VARCHAR(20)
```

### 2.5 Consultas básicas en SQL

Para comenzar a hacer consultas primero debemos llenar nuestras tablas. Asegurate de que las instancias de las tablas sean las siguientes:

#### Capitanes

| cid | cnombre     | crating | cedad |
|-----|-------------|---------|-------|
| 23  | King Arturo | 8       | 31    |
| 29  | Juan        | 1       | 33    |
| 31  | Andy        | 8       | 55    |
| 32  | Felipe      | 8.4     | 25    |
| 58  | Oscar       | 10      | 35    |
| 64  | Isidora     | 7.5     | 35    |
| 71  | Pedro       | 10      | 16    |
| 74  | Isidora     | 9       | 35    |
| 85  | Rosa        | 3       | 25    |
| 95  | Romano      | 5.5     | 63    |

#### Botes

| bid | bnombre   | bcolor |
|-----|-----------|--------|
| 101 | Catamaran | Azul   |
| 102 | Catamaran | Rojo   |
| 103 | Endurance | Verde  |
| 104 | Yate      | Rojo   |

#### Reservas

| cid | bid | fecha    |
|-----|-----|----------|
| 23  | 101 | 10/10/16 |
| 23  | 102 | 10/10/16 |
| 23  | 103 | 8/10/16  |
| 23  | 104 | 7/10/17  |
| 31  | 102 | 10/11/17 |
| 31  | 103 | 6/11/18  |
| 31  | 104 | 12/11/18 |
| 64  | 101 | 5/9/18   |
| 64  | 102 | 8/9/18   |
| 74  | 103 | 8/9/18   |

**Hint**: Para insertar un tipo fecha en SQLITE tienes que ingresarlo entre comillas simples en el formato `'YYYY-MM-DD'`, por ejemplo:

In [ ]:
%sql INSERT INTO Reservas VALUES(23, 101, '2016-10-10')

In [ ]:
# Llenar tabla Capitanes
%sql

In [ ]:
# Llenar tabla Botes
%sql

In [ ]:
# Llenar tabla Reservas
%sql

Las consultas más básicas son de la forma `SELECT - FROM - WHERE`. En general, la consulta de álgebra relacional:

$$
\pi_{a_1, \dots, a_n}(\sigma_{\text{condiciones}}(R_1 \times R_m))
$$

se traduce en SQL como:

```SQL
SELECT a_1, ..., a_n
FROM R_1, ..., R_m
WHERE <condiciones>
```

**2.** Prueba la siguiente consulta:

In [ ]:
%sql SELECT cid, cnombre, crating, cedad FROM Capitanes

¿Qué es lo que debería hacer la consulta anterior? Recuerda que si lo que quieres es proyectar todos los atributos puedes usar `*`.

**3.** Ahora realiza dos consultas, una para obtener todo desde la tabla `Reservas` y todo de la tabla `Botes`. Revisa que agregaste todos los datos correctamente.

In [ ]:
# Reservas
%sql

In [ ]:
# Botes
%sql

Ahora ejecuta la consulta a continuación. ¿Qué crees que retorna?

In [ ]:
%sql SELECT * FROM Capitanes WHERE cnombre='Juan'

Ahora es tu turno, realiza las siguientes consultas en SQL:

**3.2.** Busca los nombres de los capitanes con rating más de 4 (prueba seleccionando `*` y luego solo el atributo `cnombre`).

**Hint**: Puedes usar los filtros `x = y`, `x < y`, `x <= y`, `<>` (distinto), etc. Para los números es obvio, para los strings es orden lexicográfico.

In [ ]:
%sql

**4.** Similarmente, busca los nombres de los capitanes con rating más de 4 y cuya edad está entre 25 y 40 años.

**Ayuda**: Combina condiciones en el `WHERE` usando `AND` y `OR`. Recuerda que para negar algo puedes usar `NOT`.

In [ ]:
%sql

**5.** Busca la edad de los capitanes que no tienen una `'a'` en su nombre (recuerda usar `LIKE`).

In [ ]:
%sql

**6.** Escribe una consulta que entregue los nombres y rating de los capitanes, pero ordenado por `crating`. Para ordenar agrega un `ORDER BY (<atr_1>, ..., <atr_N>) DESC` al final de tu consulta. Eso ordena los resultados por el atributo `<atr_1>`, después `<atr_2>`, etc. Prueba que pasa al eliminar el `DESC` al final del `ORDER BY`.

In [ ]:
%sql 

### 2.6 Consultas con *joins*, DISTINCT, UNION, INTERSECT, EXCEPT, IN, BETWEEN

Imaginemos que queremos encontrar los nombres de los capitanes que reservaron el bote 103. 

**7.** Escribe primero esta consulta en Álgebra relacional.

Una forma directa para traspasarlo a SQL es haciendo un producto cartesiano pasando ambas tablas en la sentencia `FROM` y poniendo la condición del join en el `WHERE`:

In [ ]:
%%sql
SELECT  *
FROM  Capitanes, Reservas
WHERE Capitanes.cid = Reservas.cid
AND Reservas.bid = 103

Este es un buen minuto para repasar los joins. Puedes correr la consulta de arriba sin el `AND` para ver lo que sería el resultado de $\text{Capitanes} \bowtie_{\text{cid}=\text{cid}} \text{Reservas}$. ¿Qué pasa al agregar la condición del `AND`?

Ahora intenta escribir las siguientes consultas:

**8.** Los nombres de los capitanes que reservaron un bote rojo.

In [ ]:
%sql

**9.** Los nombres de los capitanes que reservaron un bote rojo o un bote verde.

In [ ]:
%sql

**10.** La misma anterior, pero usando `UNION` para unir el resultado de dos consultas. Prueba usando `UNION` y luego `UNION ALL`. ¿Cuál es la diferencia?

In [ ]:
# UNION
%sql

In [ ]:
# UNION ALL
%sql

Usa `AS` para cambiar de nombres a los atributos (esto es útil no solo para la unión!). La siguiente consulta entrega todos los ids del sistema:

In [ ]:
%%sql
SELECT cid AS id
FROM  Capitanes
UNION
SELECT bid AS id
FROM Botes

**11.** Los nombres de los capitanes que reservaron un bote rojo y un bote verde. Puedes usar `INTERSECT` para intersectar el resultado de dos consultas. ¿Puedes escribir también esta consulta usando AND?

In [ ]:
%sql

**12.** Los nombres de los capitanes que reservaron un bote rojo pero no uno verde. Puedes usar `EXCEPT` para expresar la diferencia.

In [ ]:
%sql

**13.** Los nombres de los botes que no son de color rojo, ni verde, ni amarillo, pero utilizando `IN`.
**Hint:** También debes utilizar `NOT`.

In [ ]:
%sql

**14.** Los nombres de los capitanes cuyo ranking está entre 8 y 9 inclusive, utiliando `BETWEEEN`. 

In [ ]:
%sql

**15.** Los nombres de los capitanes que reservaron un bote Azul entre el el 8 de octubre de 2016 y el 8 de octubre de 2017, utilizando `BETWEEN`.

In [ ]:
%sql

**16.** El nombre del capitán con máximo crating (podría ser más de uno).

In [ ]:
%sql

**17.** Los nombres de los capitanes que hayan reservado un bote de un color mas de una vez, junto con el color

In [ ]:
%sql